In [ ]:
%pip install pyexiftool 

#for macOS users:
#brew install exiftool  
#for Windows users, download from https://exiftool.org/ and follow installation instructions there.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import json, subprocess, shutil
from pathlib import Path
import math, folium
from folium import plugins

In [3]:

heic_path = Path("./data/IMG_1967.HEIC")

if not heic_path.exists():
    raise FileNotFoundError(f"File not found: {heic_path}")
if not shutil.which("exiftool"):
    raise RuntimeError("exiftool not found. Install it (e.g., brew install exiftool).")

cmd = ["exiftool", "-G", "-a", "-s", "-j", str(heic_path)]  # groups + all duplicates + short names + JSON
raw = subprocess.check_output(cmd, text=True)
meta = json.loads(raw)[0]

# quick keyword scan
keys = [k for k in meta.keys()]
hits = [k for k in keys if any(w in k.lower() for w in ["gyro","accel","imu","angular","velocity","camm","rotation"])]
print("Possible IMU-related tags:", hits)

# print values for any hits
for k in hits:
    print(k, "=", meta[k])


Possible IMU-related tags: ['QuickTime:Rotation', 'MakerNotes:AccelerationVector']
QuickTime:Rotation = Rotate 90 CW
MakerNotes:AccelerationVector = 0.004302905408 -0.9489275818 0.2779247464


In [4]:
def extract_gps(heic_path: Path):
    cmd = ["exiftool", "-n", "-j", "-GPSLatitude", "-GPSLongitude", str(heic_path)]
    meta = json.loads(subprocess.check_output(cmd, text=True))[0]
    lat = meta.get("GPSLatitude")
    lon = meta.get("GPSLongitude")
    if lat is None or lon is None:
        raise ValueError("No GPS in this file.")
    return float(lat), float(lon)

In [5]:
lat, lon = extract_gps(heic_path)
print("GPS:", lat, lon)

GPS: 31.2817333333333 121.503791666667


In [9]:
def parse_accel(meta):
    val = meta.get("MakerNotes:AccelerationVector") or meta.get("AccelerationVector")
    if val is None:
        raise ValueError("AccelerationVector not found in metadata.")
    if isinstance(val, str):
        parts = val.replace(",", " ").split()
        ax, ay, az = map(float, parts[:3])
    else:
        ax, ay, az = map(float, val[:3])
    return ax, ay, az

lat, lon = extract_gps(heic_path)
ax, ay, az = parse_accel(meta)

# Apple's accel axes are rotated ~45° from map north/east; rotate to align arrow with map
offset_deg = 45.0
theta = math.radians(offset_deg)
rax = ax * math.cos(theta) - ay * math.sin(theta)
ray = ax * math.sin(theta) + ay * math.cos(theta)

scale_m = 10.0  # arrow length in meters for visualization
norm = math.hypot(rax, ray) or 1e-9
ex, ey = (-(rax / norm) * scale_m), (-(ray / norm) * scale_m)

dlat = ey / 111_111
dlon = ex / (111_111 * math.cos(math.radians(lat)))
end_lat = lat + dlat
end_lon = lon + dlon

m = folium.Map(location=[lat, lon], zoom_start=18)
#folium.Marker([lat, lon], tooltip="Photo").add_to(m)
segment = folium.PolyLine([[lat, lon], [end_lat, end_lon]], color="red", weight=3, opacity=0.8).add_to(m)
plugins.PolyLineTextPath(
    segment,
    "▶",              # arrow glyph
    repeat=True,
    offset=7,
    attributes={"fill": "red", "font-weight": "bold", "font-size": "16"}
).add_to(m)
m